In [1]:
import pygame
import gym
from jumper import JumpEnv
from navigator import JetEnv

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import random
import numpy as np
from collections import deque
import os
import matplotlib.pyplot as plt
from PIL import Image
from stable_baselines3 import  PPO
import torchvision.transforms as T
from PIL import Image
from torchvision import transforms

pygame 2.1.0 (SDL 2.0.16, Python 3.8.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\Ernest\.conda\envs\PCC\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 206] The filename or extension is too long: 'c:\\Users\\Ernest\\.conda\\envs\\PCC\\lib\\site-packages\\torchvision''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
learning_rate = 5e-4
minibatch_size = 64
discount_factor = 0.99
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
gym.envs.register(
        id='JetEnv-v0',
        entry_point='__main__:JetEnv',
    )

env = gym.make('JetEnv-v0')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (500, 500, 3)
State size:  500
Number of actions:  5


In [4]:
obs = env.reset()
print(obs.shape)

(500, 500, 3)


c:\Users\Ernest\.conda\envs\PCC\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\Ernest\.conda\envs\PCC\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
c:\Users\Ernest\.conda\envs\PCC\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(


In [5]:
def preprocess_frame(frame): #frame converted to pytorch tensors
  frame = Image.fromarray(frame) #numpy array to a PIL image class as image is now in numpyarray form
  preprocess = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()]) # First resizing image and the turning it to a tensor(Also normalizes the frame).
  return preprocess(frame).unsqueeze(0) #adding extra batch dimension. (Which batch each frame belongs to)
     

In [6]:
class Network(nn.Module):

  def __init__(self, action_size, seed = 42): #constructor init method we have multidimentional array here hence, action size passed here.
    super(Network, self).__init__() # Initilizer of the base class to correctly activate inheretance
    self.seed = torch.manual_seed(seed)
    self.conv1 = nn.Conv2d(3, 32, kernel_size = 8, stride = 4) # Convulation --> 3-inputchannels(RGB), outputchannels-32, kernel size(8x8), stride = 4. -- EYES of AI
    self.bn1 = nn.BatchNorm2d(32) # Helps in training Normalization
    self.conv2 = nn.Conv2d(32, 64, kernel_size = 4, stride = 2) # These parameters are changed on the basis of experimentation and hyperparameter tuning for optimal result.
    self.bn2 = nn.BatchNorm2d(64)
    self.conv3 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1)
    self.bn3 = nn.BatchNorm2d(64)
    self.conv4 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1)
    self.bn4 = nn.BatchNorm2d(128) # EYES of AI finished.
    # Neurons in Flattening layer -- (Inputsize-Kernelsize + 2*padding)/stride + 1 -- for each convolutional layer.
    self.fc1 = nn.Linear(10 * 10 * 128, 512) # Fulllyconnected Layers - Brain of AI (inpur , Output Neurons(Taken by Experimentation))
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, action_size) # Brain Finished

  def forward(self, state): # Foreward propogation
    x = F.relu(self.bn1(self.conv1(state))) # Using conv1(Convolutional) in bn1(BatchNormalization) this in total taken in relu activation function.
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.relu(self.bn3(self.conv3(x)))
    x = F.relu(self.bn4(self.conv4(x)))
    x = x.view(x.size(0), -1) # Flattening -- first dimentions are kept and other are flattened to the first i.e 1 dimension
    x = F.relu(self.fc1(x)) # Forewarding to fully connected layer.
    x = F.relu(self.fc2(x))
    return self.fc3(x)

In [7]:
class Agent(): #Creating Agent while integrating Experience Replay and Image Preprocessing

  def __init__(self, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.action_size = action_size
    self.local_qnetwork = Network(action_size).to(self.device)
    self.target_qnetwork = Network(action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = deque(maxlen = 10000) #double ended queue maxlen = capacity

  def step(self, state, action, reward, next_state, done):
    state = preprocess_frame(state) # preprossing both the states
    next_state = preprocess_frame(next_state)
    self.memory.append((state, action, reward, next_state, done))
    if len(self.memory) > minibatch_size: #if greater than batch size then learn.
      experiences = random.sample(self.memory, k = minibatch_size) # take experiences total i.e of the batchsize
      self.learn(experiences, discount_factor) # then learn form them

  def act(self, state, epsilon = 0.):
    state = preprocess_frame(state).to(self.device) # preprossing the images
    self.local_qnetwork.eval() #eval firm nn.Module
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, actions, rewards, next_states, dones = zip(*experiences) # unzipping the experiences (all states are already mytorch tensors)
    states = torch.from_numpy(np.vstack(states)).float().to(self.device) # vstack function can take a pytorch tensor and convert it to a numpy array.(i.e we dont have to convert it to a numpy array) After this we reconvert them into Pytorch tensors
    actions = torch.from_numpy(np.vstack(actions)).long().to(self.device) # alternative could be torch.cat function
    rewards = torch.from_numpy(np.vstack(rewards)).float().to(self.device)
    next_states = torch.from_numpy(np.vstack(next_states)).float().to(self.device)
    dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(self.device)
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

In [8]:
agent = Agent(number_actions)

In [9]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 10000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 500.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

c:\Users\Ernest\.conda\envs\PCC\lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
c:\Users\Ernest\.conda\envs\PCC\lib\site-packages\gym\utils\passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


Episode 22	Average Score: -94.66

KeyboardInterrupt: 